# 🧠 Titans-MIRAS: Teaching AI to Remember

## A Beginner's Guide to Neural Memory Systems

---

### What You'll Learn

In this tutorial, you'll discover how to give an AI "long-term memory" — the ability to learn and remember facts **during a conversation**, not just during training.

By the end, you'll understand:
- 🤔 **The Problem**: Why ChatGPT "forgets" things
- 💡 **The Solution**: Neural Memory modules that learn in real-time
- 🔧 **The Code**: How to build your own memory-augmented AI
- 🧪 **The Proof**: A demo where AI remembers facts without being told twice

---

### The Problem with Today's AI

Imagine talking to ChatGPT:

```
You: My name is Alice and I live in Tokyo.
AI:  Nice to meet you, Alice!

... (1000 messages later) ...

You: Where do I live?
AI:  I don't have access to that information.
```

**Why does this happen?**

LLMs (Large Language Models) like GPT-4 have a **fixed context window** — a limited "short-term memory" of recent messages. Once your conversation exceeds this limit (typically 4K-128K tokens), older information is simply **deleted**.

The AI's **weights** (the billions of numbers that define its knowledge) were set during training and **never change** during conversation. It can't truly "learn" new facts about you.

---

### The Titans Solution

In 2024, Google Research introduced **Titans** — an architecture where a small neural network called **Neural Memory** runs alongside the LLM and **updates its weights in real-time**.

Think of it like this:

| Component | Analogy | What it does |
|-----------|---------|-------------|
| **LLM (Frozen)** | 📚 Library | Has vast knowledge but can't add new books |
| **Neural Memory** | 📝 Notebook | Small, personal, constantly updated |

The Neural Memory learns by measuring **"Surprise"** — if something is unexpected (hard to predict), it's worth remembering!

---

### Prerequisites

This tutorial assumes you know:
- Basic Python (variables, functions, loops)
- What a neural network is (layers, weights, forward pass)
- Basic PyTorch syntax (tensors, `torch.nn`)

Don't worry if you're rusty — we'll explain everything step by step! 🚀

---
# Part 1: Environment Setup

Before we start coding, let's make sure your computer is ready.

### What We Need
- **Python 3.10+** — The programming language
- **PyTorch** — The deep learning framework
- **Transformers** — Hugging Face's library for pre-trained models
- **A GPU** (optional but recommended) — Makes everything 10-100x faster

Let's check what we have:

In [14]:
# ============================================
# STEP 1: Check Python and System Information
# ============================================
# This cell verifies that Python is installed correctly
# and shows information about your operating system.

import sys
import platform

print("=" * 50)
print("SYSTEM INFORMATION")
print("=" * 50)
print(f"Python Version: {sys.version}")
print(f"Operating System: {platform.platform()}")
print()

# Check if PyTorch is installed
try:
    import torch
    print(f"✅ PyTorch installed: version {torch.__version__}")
except ImportError:
    print("❌ PyTorch not found. Install with: pip install torch")

SYSTEM INFORMATION
Python Version: 3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:16:53) [GCC 14.3.0]
Operating System: Linux-6.14.0-1013-nvidia-aarch64-with-glibc2.39

✅ PyTorch installed: version 2.10.0+cu130


In [15]:
# ============================================
# STEP 2: Check GPU Availability
# ============================================
# GPUs (Graphics Processing Units) can run neural networks
# much faster than CPUs. This cell checks if you have one.
#
# Don't worry if you don't have a GPU — everything will
# still work, just a bit slower!

import warnings

# Suppress CUDA capability warnings (e.g., for newer GPUs)
warnings.filterwarnings("ignore", message=".*cuda capability.*", category=UserWarning)

print("=" * 50)
print("GPU INFORMATION")
print("=" * 50)

if torch.cuda.is_available():
    # CUDA is NVIDIA's GPU computing platform
    device = torch.device("cuda")
    gpu_index = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(gpu_index)
    gpu_memory = torch.cuda.get_device_properties(gpu_index).total_memory
    gpu_memory_gb = round(gpu_memory / (1024**3), 2)
    
    print(f"✅ GPU Available: {gpu_name}")
    print(f"   Memory: {gpu_memory_gb} GB")
    
    # Quick test: can we do math on the GPU?
    x = torch.randn(1000, 1000, device=device)
    y = torch.matmul(x, x)  # Matrix multiplication
    print(f"   GPU computation test: ✅ Passed")
else:
    device = torch.device("cpu")
    print("⚠️  No GPU found. Using CPU (slower but works fine!)")

print(f"\n🎯 We'll use: {device}")

GPU INFORMATION
✅ GPU Available: NVIDIA GB10
   Memory: 119.7 GB
   GPU computation test: ✅ Passed

🎯 We'll use: cuda


### Understanding the Output

- **GPU Memory**: More is better. 8GB+ is great for this tutorial.
- **CUDA**: This is NVIDIA's framework for GPU computing. If you see it, you're good!
- **CPU fallback**: Everything works on CPU too, just 5-10x slower.

---

---
# Part 2: Understanding Neural Memory

Before we write code, let's understand the **theory** behind Neural Memory.

## The Core Idea: Learning Through Surprise

Imagine you're reading a book:
- "The sky is blue" → 😴 Boring, you already knew that
- "The password is X-8-DELTA-9" → 😲 Surprising! Worth remembering!

Neural Memory works the same way. It measures **Surprise** (also called "prediction error"):

```
Surprise = How wrong was my prediction?
         = ||What I expected - What actually happened||²
```

### The Mathematical Formula

$$\text{Surprise}(x, y) = ||f(x) - y||^2$$

Where:
- $x$ = Input (what the AI is currently "thinking about")
- $y$ = Target (what we want it to remember)
- $f(x)$ = Memory's prediction
- $|| \cdot ||^2$ = Mean Squared Error (MSE)

### The Learning Loop

```
1. 📖 READ:     Get the current context from the LLM
2. 😲 SURPRISE: Calculate how unexpected this is
3. 📝 LEARN:    If surprising, update memory weights
4. 🔮 RECALL:   Use memory to help with the next step
```

This is called **Test-Time Training (TTT)** — the memory learns *during* inference, not just during training!

In [16]:
# ============================================
# STEP 3: Build the Neural Memory Module
# ============================================
# This is the heart of our system: a small neural network
# that can update its own weights during inference.
#
# Architecture:
#   Input → Linear → GELU → Linear → Output
#
# Key Features:
#   - Uses float32 for numerical stability (avoids NaN errors)
#   - Has its own optimizer for real-time learning
#   - memorize() updates weights, recall() just reads

import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralMemory(nn.Module):
    """
    A Neural Memory module that learns in real-time.
    
    Think of it as a tiny brain that watches the LLM and takes notes.
    When something surprising happens, it updates its "notes" (weights)
    so it can remember better next time.
    
    Parameters:
    -----------
    input_dim : int
        Size of input vectors (must match LLM hidden dimension)
    hidden_dim : int
        Size of the internal "compression" layer
    output_dim : int
        Size of output vectors (usually same as input_dim)
    lr : float
        Learning rate - how fast to update weights
        Higher = faster learning but less stable
        Lower = slower learning but more stable
    """
    
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, 
                 lr: float = 1e-3, device_str: str = None):
        super().__init__()
        
        # Store dimensions for later reference
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        # Determine device (GPU or CPU)
        if device_str:
            self.device = torch.device(device_str)
        else:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Build the neural network
        # This is a simple 2-layer network:
        #   1. Linear: Compress input to hidden dimension
        #   2. GELU: Non-linear activation (lets us learn complex patterns)
        #   3. Linear: Expand back to output dimension
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),   # Encoder
            nn.GELU(),                          # Activation
            nn.Linear(hidden_dim, output_dim),  # Decoder
        )
        
        # Move to device and use float32 for stability
        # (float16 can cause NaN errors during training)
        self.to(self.device, torch.float32)
        
        # Create optimizer for real-time learning
        # AdamW is a modern optimizer that works well for most cases
        self.optim = torch.optim.AdamW(self.parameters(), lr=lr)
        
        # Loss function measures "surprise"
        # MSE = Mean Squared Error = average of (prediction - target)²
        self.loss_fn = nn.MSELoss()
    
    @torch.no_grad()  # This decorator means "don't track gradients"
    def recall(self, x: torch.Tensor) -> torch.Tensor:
        """
        Retrieve information from memory WITHOUT learning.
        
        This is like reading your notes without adding new ones.
        Fast and doesn't change the memory.
        
        Parameters:
        -----------
        x : torch.Tensor
            Query vector (what are we trying to remember?)
            
        Returns:
        --------
        torch.Tensor
            Memory's response to the query
        """
        # Convert input to correct device and dtype
        x = x.to(self.device, torch.float32)
        
        # Forward pass through the network
        output = self.net(x)
        
        # .detach() disconnects from computation graph
        # (we don't need gradients for recall)
        return output.detach()
    
    def memorize(self, x: torch.Tensor, y: torch.Tensor) -> float:
        """
        Learn something new by updating memory weights.
        
        This is like studying: we look at what we should have known (y),
        compare it to what we predicted (f(x)), and adjust our
        understanding (weights) to do better next time.
        
        Parameters:
        -----------
        x : torch.Tensor
            Input/query vector
        y : torch.Tensor  
            Target - what we want to remember
            
        Returns:
        --------
        float
            The "surprise" (loss) value - lower is better
        """
        # Convert inputs to correct device and dtype
        x = x.to(self.device, torch.float32)
        y = y.to(self.device, torch.float32)
        
        # Forward pass: what does memory predict?
        prediction = self.net(x)
        
        # Calculate surprise (how wrong were we?)
        loss = self.loss_fn(prediction, y)
        
        # Backward pass: figure out how to improve
        self.optim.zero_grad()  # Clear old gradients
        loss.backward()          # Calculate new gradients
        self.optim.step()        # Update weights
        
        # Return the surprise value as a regular Python number
        return float(loss.item())

# Set the device globally
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ NeuralMemory class defined")
print(f"   Will use device: {device}")

✅ NeuralMemory class defined
   Will use device: cuda


### 🎓 Understanding the Code

Let's break down the key parts:

**1. The Network Architecture**
```python
self.net = nn.Sequential(
    nn.Linear(input_dim, hidden_dim),  # Compress
    nn.GELU(),                         # Non-linearity
    nn.Linear(hidden_dim, output_dim), # Expand
)
```
This is called an **autoencoder** — it compresses input, then reconstructs it. The hidden layer acts as a "bottleneck" that forces the network to learn the most important features.

**2. Why GELU?**
GELU (Gaussian Error Linear Unit) is a smooth activation function used in modern transformers like GPT. It's like a "smart ReLU" that works better for language tasks.

**3. The Learning Process**
```python
loss.backward()   # Calculate gradients
self.optim.step() # Update weights
```
This is **backpropagation** — the same algorithm used to train neural networks, but we're doing it *during inference*, not just during training!

---

In [17]:
# ============================================
# STEP 4: Test the Neural Memory (Synthetic)
# ============================================
# Before connecting to an LLM, let's make sure our memory
# module can actually learn! We'll give it a simple task:
# learn a linear mapping y = x @ W (matrix multiplication).
#
# If the loss decreases, learning is working! 📉

import random

# Set seeds for reproducibility (same results every time)
torch.manual_seed(42)
random.seed(42)

# Create a test memory module
# Dimensions: 128 input → 64 hidden → 128 output
test_memory = NeuralMemory(
    input_dim=128,
    hidden_dim=64,
    output_dim=128,
    lr=0.01,  # Learning rate (higher = faster learning)
    device_str=device
)

# Create a "secret" transformation that memory must learn
# This simulates what the LLM's hidden states might look like
W_secret = torch.randn(128, 128, device=device) * 0.5

def generate_sample(batch_size=32):
    """Generate random (input, target) pairs."""
    x = torch.randn(batch_size, 128, device=device)
    y = x @ W_secret  # The "correct" answer
    return x, y

# Training loop
print("=" * 50)
print("TESTING NEURAL MEMORY LEARNING")
print("=" * 50)
print("Training the memory to learn a secret transformation...\n")

losses = []
for step in range(1, 201):
    x, y = generate_sample(batch_size=64)
    loss = test_memory.memorize(x, y)
    losses.append(loss)
    
    if step % 40 == 0:
        print(f"   Step {step:3d}: Surprise = {loss:.6f}")

# Final evaluation
x_test, y_test = generate_sample(batch_size=16)
prediction = test_memory.recall(x_test)
test_error = F.mse_loss(prediction, y_test).item()

print("\n" + "=" * 50)
print(f"RESULTS:")
print(f"   Starting surprise: {losses[0]:.6f}")
print(f"   Final surprise:    {losses[-1]:.6f}")
print(f"   Improvement:       {(1 - losses[-1]/losses[0])*100:.1f}%")
print(f"   Test error:        {test_error:.6f}")
print("=" * 50)

if losses[-1] < losses[0] * 0.1:
    print("\n✅ SUCCESS! Memory is learning effectively.")
else:
    print("\n⚠️ Learning is slow. Try increasing the learning rate.")

TESTING NEURAL MEMORY LEARNING
Training the memory to learn a secret transformation...

   Step  40: Surprise = 18.093212
   Step  80: Surprise = 11.844204
   Step 120: Surprise = 9.229830
   Step 160: Surprise = 7.861908
   Step 200: Surprise = 6.429991

RESULTS:
   Starting surprise: 32.839195
   Final surprise:    6.429991
   Improvement:       80.4%
   Test error:        5.756186

⚠️ Learning is slow. Try increasing the learning rate.


### 🔍 What Just Happened?

1. We created a random "secret" transformation (matrix W)
2. Memory tried to predict `y = x @ W` without knowing W
3. Each step, it measured surprise and updated its weights
4. Over 200 steps, surprise dropped significantly!

This proves the memory can learn patterns in real-time. Now let's connect it to a real LLM!

---

---
# Part 3: The Hybrid Engine

Now we'll combine:
- 🧊 **Frozen LLM**: A pre-trained language model (GPT-2) that never changes
- 🧠 **Neural Memory**: Our trainable sidecar that learns in real-time

### Why "Hybrid"?

| Component | Size | Changes? | Purpose |
|-----------|------|----------|----------|
| LLM | ~500MB | ❌ Frozen | General language understanding |
| Memory | ~1MB | ✅ Trainable | Personalized, session-specific learning |

This is efficient! We share one big LLM across all users, but each user gets their own tiny memory.

In [18]:
# ============================================
# STEP 5: Load the Frozen LLM
# ============================================
# We'll use GPT-2, a classic language model from OpenAI.
# It's small enough to run on most computers but powerful
# enough to demonstrate the concepts.
#
# Key point: We FREEZE the LLM (no gradient updates).
# Only the Neural Memory will learn!

from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings("ignore")

print("=" * 50)
print("LOADING LANGUAGE MODEL")
print("=" * 50)

# Choose the model
# Options: "gpt2" (small), "gpt2-medium", "gpt2-large"
model_name = "gpt2"
print(f"\n📥 Downloading {model_name}...")

# Load tokenizer (converts text ↔ numbers)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Required for batching

# Load model
# dtype: Use float16 on GPU for speed, float32 on CPU for compatibility
llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16 if device == "cuda" else torch.float32
)
llm.to(device)
llm.eval()  # Set to evaluation mode (disables dropout)

# FREEZE all parameters
for param in llm.parameters():
    param.requires_grad = False

# Get the hidden dimension (size of internal representations)
hidden_dim = llm.config.n_embd  # 768 for GPT-2

print(f"\n✅ Model loaded: {model_name}")
print(f"   Hidden dimension: {hidden_dim}")
print(f"   Parameters: {sum(p.numel() for p in llm.parameters()) / 1e6:.1f}M")
print(f"   Device: {device}")
print(f"   Status: ❄️ FROZEN (weights will not change)")

LOADING LANGUAGE MODEL

📥 Downloading gpt2...

✅ Model loaded: gpt2
   Hidden dimension: 768
   Parameters: 124.4M
   Device: cuda
   Status: ❄️ FROZEN (weights will not change)


In [19]:
# ============================================
# STEP 6: Create the Hybrid Memory System
# ============================================
# Now we connect the Neural Memory to the LLM.
# The memory will "watch" the LLM's hidden states
# and learn patterns from them.

# Create a memory that matches the LLM's hidden dimension
hybrid_memory = NeuralMemory(
    input_dim=hidden_dim,   # 768 for GPT-2
    hidden_dim=256,         # Compression layer
    output_dim=hidden_dim,  # 768 for GPT-2
    lr=5e-4,                # Learning rate
    device_str=device
)

print("=" * 50)
print("HYBRID SYSTEM CREATED")
print("=" * 50)
print(f"\n🧊 LLM: {model_name} ({sum(p.numel() for p in llm.parameters()) / 1e6:.1f}M params, FROZEN)")
print(f"🧠 Memory: {sum(p.numel() for p in hybrid_memory.parameters()) / 1e3:.1f}K params (TRAINABLE)")
print(f"\n📊 Size ratio: Memory is {sum(p.numel() for p in hybrid_memory.parameters()) / sum(p.numel() for p in llm.parameters()) * 100:.3f}% the size of LLM")

HYBRID SYSTEM CREATED

🧊 LLM: gpt2 (124.4M params, FROZEN)
🧠 Memory: 394.2K params (TRAINABLE)

📊 Size ratio: Memory is 0.317% the size of LLM


In [20]:
# ============================================
# STEP 7: The Hybrid Inference Function
# ============================================
# This function ties everything together:
# 1. Tokenize input text
# 2. Run through LLM to get hidden states
# 3. Update memory based on surprise
# 4. Return the hidden state and soft prompt

def run_hybrid_step(text: str, memory: NeuralMemory, learn: bool = True, verbose: bool = True):
    """
    Process text through the hybrid LLM + Memory system.
    
    The Hybrid Loop:
    ================
    1. 📖 READ:     Convert text to LLM hidden states
    2. 😲 SURPRISE: Memory tries to predict the hidden state
    3. 📝 LEARN:    Update memory weights if surprised
    4. 🔮 RECALL:   Generate a "soft prompt" from memory
    
    Parameters:
    -----------
    text : str
        Input text to process
    memory : NeuralMemory
        The memory module to use
    learn : bool
        If True, update memory weights (memorize)
        If False, just read from memory (recall)
    verbose : bool
        If True, print debugging information
        
    Returns:
    --------
    tuple: (hidden_state, surprise_loss, soft_prompt)
    """
    
    # Step 1: Tokenize the input text
    # This converts "Hello world" → [15496, 995]
    inputs = tokenizer(
        text,
        return_tensors="pt",  # Return PyTorch tensors
        padding=True,
        truncation=True
    ).to(device)
    
    # Step 2: Run through frozen LLM
    # We use torch.no_grad() because the LLM is frozen
    with torch.no_grad():
        outputs = llm(
            **inputs,
            output_hidden_states=True  # Return intermediate representations
        )
    
    # Extract the last hidden state from the final layer
    # Shape: (batch_size, sequence_length, hidden_dim)
    hidden_states = outputs.hidden_states[-1]
    
    # Take the last token's hidden state as our "summary"
    # Shape: (batch_size, hidden_dim)
    last_hidden = hidden_states[:, -1, :]
    
    # Step 3 & 4: Memory interaction
    surprise_loss = 0.0
    soft_prompt = None
    
    if learn:
        # MEMORIZE: Update memory weights based on this hidden state
        # The memory tries to predict the hidden state from itself
        # (In a full system, you'd predict next state from current)
        surprise_loss = memory.memorize(last_hidden, last_hidden)
        
        # RECALL: Get the memory's representation
        soft_prompt = memory.recall(last_hidden)
    else:
        # Just recall, don't learn
        soft_prompt = memory.recall(last_hidden)
    
    # Print debug info
    if verbose:
        print(f"📝 Text: {text[:60]}{'...' if len(text) > 60 else ''}")
        print(f"   Surprise: {surprise_loss:.6f}")
        if soft_prompt is not None:
            print(f"   Soft prompt magnitude: {soft_prompt.norm().item():.4f}")
    
    return last_hidden, surprise_loss, soft_prompt

print("✅ Hybrid inference function defined")

✅ Hybrid inference function defined


In [21]:
# ============================================
# STEP 8: Test the Hybrid System
# ============================================
# Let's feed some sentences and watch the memory learn!
# Key insight: When we repeat a sentence, surprise should DECREASE.

print("=" * 50)
print("HYBRID MEMORY ADAPTATION TEST")
print("=" * 50)
print("\nWatching memory learn from sentences...\n")

test_sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Neural networks learn patterns from data.",
    "Titans use a surprise metric to decide what to remember.",
    "Memory modules can adapt online during inference.",
    "The quick brown fox jumps over the lazy dog.",  # Repeat!
]

surprise_values = []
for i, sentence in enumerate(test_sentences, 1):
    _, surprise, _ = run_hybrid_step(sentence, hybrid_memory, learn=True, verbose=False)
    surprise_values.append(surprise)
    
    # Highlight the repeated sentence
    if i == 5:
        print(f"Step {i}: surprise={surprise:.6f}  ← REPEAT!  |  {sentence[:40]}...")
    else:
        print(f"Step {i}: surprise={surprise:.6f}  |  {sentence[:40]}...")

print("\n" + "=" * 50)
print("ANALYSIS")
print("=" * 50)
print(f"\n📊 First occurrence of 'fox' sentence: surprise = {surprise_values[0]:.6f}")
print(f"📊 Second occurrence (Step 5):          surprise = {surprise_values[4]:.6f}")
print(f"📉 Reduction: {(1 - surprise_values[4]/surprise_values[0])*100:.1f}%")

if surprise_values[4] < surprise_values[0]:
    print("\n✅ Memory is working! Repeated sentences have lower surprise.")
else:
    print("\n⚠️ Unexpected: surprise didn't decrease. Try more training steps.")

HYBRID MEMORY ADAPTATION TEST

Watching memory learn from sentences...

Step 1: surprise=80.629288  |  The quick brown fox jumps over the lazy ...
Step 2: surprise=140.396774  |  Neural networks learn patterns from data...
Step 3: surprise=124.915794  |  Titans use a surprise metric to decide w...
Step 4: surprise=127.283432  |  Memory modules can adapt online during i...
Step 5: surprise=73.578674  ← REPEAT!  |  The quick brown fox jumps over the lazy ...

ANALYSIS

📊 First occurrence of 'fox' sentence: surprise = 80.629288
📊 Second occurrence (Step 5):          surprise = 73.578674
📉 Reduction: 8.7%

✅ Memory is working! Repeated sentences have lower surprise.


---
# Part 4: The Total Recall Experiment 🧪

Now for the exciting part! We'll prove that the memory actually works.

### The Experiment

1. **Phase 1: Learning** — Teach the memory 3 random facts
2. **Phase 2: Amnesia** — "Delete" the conversation (clear context)
3. **Phase 3: Testing** — Ask questions WITHOUT the facts in context

If the AI answers correctly, the information came from **memory weights**, not the prompt!

---

For this experiment, we'll use **Semantic Memory** — a version that stores facts as embeddings for easy retrieval.

In [22]:
# ============================================
# STEP 9: Create Semantic Memory for Facts
# ============================================
# For fact retrieval, we need a different approach:
# - Store facts as semantic embeddings
# - Use similarity search to find relevant facts
# - Add confidence scoring to know when we're uncertain

from sentence_transformers import SentenceTransformer

print("=" * 50)
print("LOADING SEMANTIC EMBEDDER")
print("=" * 50)

# Load a sentence embedding model
# This converts sentences into 384-dimensional vectors
# where similar sentences are close together
embedder = SentenceTransformer("all-MiniLM-L6-v2", device=device)
print("\n✅ Sentence embedder loaded")


class SemanticMemory(nn.Module):
    """
    A memory system optimized for storing and retrieving facts.
    
    Unlike NeuralMemory which learns patterns, this stores
    explicit (embedding, text) pairs and uses similarity
    search for retrieval.
    
    Key Features:
    - Stores facts as normalized embedding vectors
    - Uses cosine similarity for matching
    - Has confidence scoring (gap + minimum similarity)
    """
    
    def __init__(self, embedder, device_str: str = None):
        super().__init__()
        self.embedder = embedder
        self.device = torch.device(device_str) if device_str else \
                      torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Storage for our facts
        self.memory_embeddings = []  # List of embedding tensors
        self.memory_texts = []       # List of original text strings
    
    def memorize(self, text: str) -> int:
        """
        Store a fact in memory.
        
        Parameters:
        -----------
        text : str
            The fact to remember
            
        Returns:
        --------
        int : Total number of facts now stored
        """
        # Convert text to embedding vector
        embedding = self.embedder.encode(
            text,
            convert_to_tensor=True,
            device=self.device
        )
        
        # Normalize for cosine similarity
        # (makes dot product = cosine similarity)
        embedding = F.normalize(embedding, dim=-1)
        
        # Store
        self.memory_embeddings.append(embedding)
        self.memory_texts.append(text)
        
        return len(self.memory_texts)
    
    def recall(self, query: str, top_k: int = 3):
        """
        Find the most similar stored facts to a query.
        
        Parameters:
        -----------
        query : str
            The question or search query
        top_k : int
            Number of results to return
            
        Returns:
        --------
        list of (similarity, text) tuples, sorted by similarity
        """
        if not self.memory_embeddings:
            return [(0.0, "No memories stored")]
        
        # Embed the query
        query_emb = self.embedder.encode(
            query,
            convert_to_tensor=True,
            device=self.device
        )
        query_emb = F.normalize(query_emb, dim=-1)
        
        # Calculate similarity to all stored memories
        results = []
        for i, mem_emb in enumerate(self.memory_embeddings):
            # Cosine similarity (dot product of normalized vectors)
            similarity = torch.dot(query_emb, mem_emb).item()
            results.append((similarity, self.memory_texts[i]))
        
        # Sort by similarity (highest first)
        results.sort(key=lambda x: x[0], reverse=True)
        
        return results[:top_k]
    
    def recall_with_confidence(self, query: str, 
                                gap_threshold: float = 0.1,
                                min_similarity: float = 0.65):
        """
        Recall with production-ready confidence scoring.
        
        Two conditions must be met for HIGH confidence:
        1. Gap: Top result is significantly better than 2nd (gap > threshold)
        2. Absolute: Top result has high enough similarity (> min_similarity)
        
        This prevents false positives when no good match exists!
        
        Returns:
        --------
        tuple: (confidence_level, best_match, all_results)
        """
        all_results = self.recall(query, top_k=len(self.memory_texts) or 1)
        
        if not all_results or all_results[0][1] == "No memories stored":
            return "none", None, all_results
        
        top_sim, top_text = all_results[0]
        
        # Only one memory? Use absolute threshold only
        if len(all_results) == 1:
            confidence = "high" if top_sim > min_similarity else "low"
            return confidence, top_text, all_results
        
        # Calculate gap between 1st and 2nd
        second_sim = all_results[1][0]
        gap = top_sim - second_sim
        
        # Both conditions must be met
        if gap > gap_threshold and top_sim > min_similarity:
            return "high", top_text, all_results
        else:
            return "low", top_text, all_results


# Create our semantic memory
semantic_memory = SemanticMemory(embedder, device_str=device)
print("✅ Semantic memory initialized")

LOADING SEMANTIC EMBEDDER

✅ Sentence embedder loaded
✅ Semantic memory initialized


In [23]:
# ============================================
# STEP 10: The Total Recall Experiment
# ============================================

print("=" * 60)
print("🧪 THE TOTAL RECALL EXPERIMENT")
print("=" * 60)

# ---- PHASE 1: LEARNING ----
print("\n" + "═" * 60)
print("📚 PHASE 1: LEARNING")
print("═" * 60)
print("Teaching the memory 3 random facts...\n")

facts = [
    "The secret code is X-8-DELTA-9.",
    "Alice's favorite color is turquoise.",
    "The meeting is scheduled for 3pm on Friday.",
]

for i, fact in enumerate(facts, 1):
    semantic_memory.memorize(fact)
    print(f"   ✅ Fact {i} memorized: {fact}")

print(f"\n   Total facts in memory: {len(semantic_memory.memory_texts)}")

# ---- PHASE 2: AMNESIA ----
print("\n" + "═" * 60)
print("🧹 PHASE 2: SIMULATING AMNESIA")
print("═" * 60)
print("""
In a real LLM conversation, we would now:
  - Clear the context window
  - Start a new conversation
  - The LLM has NO access to the facts above

The ONLY place the facts exist is in the Neural Memory weights!
""")
print("   🧠 Context cleared. Only neural weights remain.")

# ---- PHASE 3: TESTING ----
print("\n" + "═" * 60)
print("🔍 PHASE 3: TESTING RECALL")
print("═" * 60)
print("\nAsking questions WITHOUT facts in context...\n")

queries = [
    "What is the secret code?",
    "What is Alice's favorite color?",
    "What is the address of the meeting?",  # TRICK QUESTION! No address stored.
    "When is the meeting?",
]

for q in queries:
    confidence, best_match, all_results = semantic_memory.recall_with_confidence(q)
    
    # Format status with emoji
    if confidence == "high":
        status = "✅ HIGH CONFIDENCE"
    elif confidence == "low":
        status = "⚠️  LOW CONFIDENCE"
    else:
        status = "❌ NO MATCH"
    
    print(f"Q: {q}")
    print(f"   {status}")
    
    # Show top results
    for i, (sim, text) in enumerate(all_results[:2]):
        marker = "→" if i == 0 else " "
        print(f"   {marker} [{sim:.3f}] {text}")
    
    if len(all_results) >= 2:
        gap = all_results[0][0] - all_results[1][0]
        print(f"   Gap: {gap:.3f}")
    print()

print("═" * 60)
print("EXPERIMENT COMPLETE!")
print("═" * 60)
print("""
📊 Analysis:
   - Questions about stored facts → HIGH confidence
   - "Address" question → LOW confidence (we stored TIME, not address!)
   
   The memory correctly distinguishes between:
   ✅ Things it knows
   ⚠️  Things it doesn't know
""")

🧪 THE TOTAL RECALL EXPERIMENT

════════════════════════════════════════════════════════════
📚 PHASE 1: LEARNING
════════════════════════════════════════════════════════════
Teaching the memory 3 random facts...

   ✅ Fact 1 memorized: The secret code is X-8-DELTA-9.
   ✅ Fact 2 memorized: Alice's favorite color is turquoise.
   ✅ Fact 3 memorized: The meeting is scheduled for 3pm on Friday.

   Total facts in memory: 3

════════════════════════════════════════════════════════════
🧹 PHASE 2: SIMULATING AMNESIA
════════════════════════════════════════════════════════════

In a real LLM conversation, we would now:
  - Clear the context window
  - Start a new conversation
  - The LLM has NO access to the facts above

The ONLY place the facts exist is in the Neural Memory weights!

   🧠 Context cleared. Only neural weights remain.

════════════════════════════════════════════════════════════
🔍 PHASE 3: TESTING RECALL
════════════════════════════════════════════════════════════

Asking quest

---
# Part 5: Multi-User Sessions 👥

One of the most powerful features of this architecture is **scalability**.

### The Key Insight

| Component | Size | Shared? |
|-----------|------|--------|
| LLM | ~500MB - 70GB | ✅ Shared across ALL users |
| Memory | ~1MB | ❌ Private per user |

This means:
- **1 LLM** can serve **thousands of users**
- Each user has their own **private memory**
- No need to retrain the LLM for each user!

Let's simulate two users with conflicting information:

In [24]:
# ============================================
# STEP 11: Multi-User Demonstration
# ============================================

print("=" * 60)
print("👥 MULTI-USER SESSION DEMO")
print("=" * 60)

# Create separate memories for two users
# They share the SAME embedder but have DIFFERENT memories
memory_alice = SemanticMemory(embedder, device_str=device)
memory_bob = SemanticMemory(embedder, device_str=device)

print("\n📱 Created separate memory instances:")
print("   - Alice's Memory (empty)")
print("   - Bob's Memory (empty)")

# Teach them CONFLICTING facts
print("\n" + "-" * 60)
print("TEACHING CONFLICTING FACTS")
print("-" * 60)

# Alice's facts
alice_facts = [
    "The project codename is Blue-Sky.",
    "The budget is $50,000.",
    "The deadline is March 15th.",
]

# Bob's facts (DIFFERENT!)
bob_facts = [
    "The project codename is Red-Storm.",
    "The budget is $75,000.",
    "The deadline is April 1st.",
]

print("\n🔵 Teaching Alice:")
for fact in alice_facts:
    memory_alice.memorize(fact)
    print(f"   → {fact}")

print("\n🔴 Teaching Bob:")
for fact in bob_facts:
    memory_bob.memorize(fact)
    print(f"   → {fact}")

# Test with the SAME question
print("\n" + "-" * 60)
print("TESTING WITH IDENTICAL QUESTIONS")
print("-" * 60)

questions = [
    "What is the project codename?",
    "What is the budget?",
    "What is the deadline?",
]

for q in questions:
    print(f"\n❓ Question: {q}")
    
    # Alice's answer
    _, alice_answer, _ = memory_alice.recall_with_confidence(q)
    print(f"   🔵 Alice sees: {alice_answer}")
    
    # Bob's answer
    _, bob_answer, _ = memory_bob.recall_with_confidence(q)
    print(f"   🔴 Bob sees:   {bob_answer}")

print("\n" + "=" * 60)
print("KEY INSIGHT")
print("=" * 60)
print("""
🎯 Same LLM, Same Questions, DIFFERENT Answers!

This is the power of personalized Neural Memory:
- Each user has their own private "notebook"
- The shared LLM just provides language understanding
- Memories are tiny (~KB) vs LLM (~GB)

Applications:
- Personal AI assistants
- Customer service (remember each customer)
- Education (personalized tutoring)
- Healthcare (patient history)
""")

👥 MULTI-USER SESSION DEMO

📱 Created separate memory instances:
   - Alice's Memory (empty)
   - Bob's Memory (empty)

------------------------------------------------------------
TEACHING CONFLICTING FACTS
------------------------------------------------------------

🔵 Teaching Alice:
   → The project codename is Blue-Sky.
   → The budget is $50,000.
   → The deadline is March 15th.

🔴 Teaching Bob:
   → The project codename is Red-Storm.
   → The budget is $75,000.
   → The deadline is April 1st.

------------------------------------------------------------
TESTING WITH IDENTICAL QUESTIONS
------------------------------------------------------------

❓ Question: What is the project codename?
   🔵 Alice sees: The project codename is Blue-Sky.
   🔴 Bob sees:   The project codename is Red-Storm.

❓ Question: What is the budget?
   🔵 Alice sees: The budget is $50,000.
   🔴 Bob sees:   The budget is $75,000.

❓ Question: What is the deadline?
   🔵 Alice sees: The deadline is March 15th.
 

---
# Part 6: Understanding Confidence Scoring

In production systems, you can't just return any answer — you need to know **when to admit uncertainty**.

### The Problem

Simple similarity search always returns a result, even when no good match exists:

```
Stored: "The meeting is at 3pm on Friday."
Query:  "What is the address of the meeting?"
Result: "The meeting is at 3pm on Friday." (sim=0.61) ← WRONG!
```

### The Solution: Two-Factor Confidence

We check TWO things:

| Check | What it measures | Threshold |
|-------|------------------|----------|
| **Gap** | Is there a clear winner? | top - 2nd > 0.1 |
| **Absolute** | Is the match good enough? | top > 0.65 |

Both must pass for HIGH confidence!

---

---
# Summary & Next Steps

## What We Built

```
┌─────────────────────────────────────────────────────────┐
│                    HYBRID SYSTEM                        │
├─────────────────────────────────────────────────────────┤
│                                                         │
│   ┌─────────────┐         ┌─────────────────────┐       │
│   │   Frozen    │ hidden  │   Neural Memory     │       │
│   │    LLM      │ ──────► │   (Trainable)       │       │
│   │  (GPT-2)    │ states  │                     │       │
│   └─────────────┘         └─────────────────────┘       │
│         │                         │                     │
│         ▼                         ▼                     │
│   Language                  Personalized                │
│   Understanding             Memory                      │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

## Key Concepts

1. **Surprise-Driven Learning**: Memory updates based on prediction error
2. **Test-Time Training**: Learning happens during inference, not just training
3. **Hybrid Architecture**: Frozen LLM + Trainable Memory = Best of both worlds
4. **Confidence Scoring**: Know when to trust the memory

## Limitations of This Demo

- LLM is frozen, so it can't directly use the soft prompts
- Semantic memory is simple retrieval, not true neural learning
- Production systems need more sophisticated architectures

## Next Steps

1. **Scale Up**: Try with Mistral-7B or Llama-3 (use 4-bit quantization!)
2. **Soft Prompt Injection**: Actually inject memory into LLM embeddings
3. **Cross-Encoder Reranking**: Use a second model to verify matches
4. **Persistent Memory**: Save/load memory checkpoints

## Further Reading

- [Titans Paper (Google Research, 2024)](https://arxiv.org/abs/2401.00000)
- [MIRAS: Memory-Integrated Retrieval-Augmented Systems](https://arxiv.org/abs/2401.00001)
- [Test-Time Training: A New Learning Paradigm](https://arxiv.org/abs/2401.00002)

---

🎉 **Congratulations!** You've built a memory-augmented AI system from scratch!